In [1]:
from IPython.display import Image

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

In [5]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [6]:
#연습용 데이터 불러오기
(train_x, train_y), (val_x, val_y) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [12]:
#흑백 이미지 0~255 값 보기
print(train_x[0][8])

[  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241
   0   0   0   0   0   0   0   0   0   0]


In [14]:
#Label 데이터 
print(val_y[0:9])

[7 2 1 0 4 1 4 9 5]


In [16]:
print('테스트 데이터 샘플 수는 '+str(val_x.shape[0])+' 개 이다.')
print('모든 데이터는 '+str(val_x.shape[1])+' * '+str(val_x.shape[2])+' 이미지다.')

테스트 데이터 샘플 수는 10000 개 이다.
모든 데이터는 28 * 28 이미지다.


In [17]:
#이미지 데이터 행렬 바꾸기
import numpy as np
train_x = np.reshape(train_x, (60000, 28, 28, 1))
val_x = np.reshape(val_x, (10000, 28, 28, 1))

print(train_x.shape)
print(val_x.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [18]:
#데이터 정규화
train_x = train_x.astype('float32')
val_x = val_x.astype('float32')

gray_scale = 255
train_x = train_x / gray_scale
val_x = val_x / gray_scale

In [19]:
#Label one hot encoding
num_classes = 10
train_y = tf.keras.utils.to_categorical(train_y, num_classes)
val_y = tf.keras.utils.to_categorical(val_y, num_classes)

In [20]:
Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/practice_cnn.png", width=800, height=200)

In [23]:
model = Sequential()
#이미지 Conv2D
model.add(Conv2D(16, kernel_size=(5,5),  #커널 사이즈 5*5
          activation='relu', #활성화 함수 relu
          input_shape=(28,28,1), padding='same'))
#이미지 각 타일의 최대값만 뽑아 행렬 축소
model.add(MaxPooling2D(pool_size=(2,2)))
#축소된 이미지 행렬 Conv
model.add(Conv2D(32, kernel_size=(5,5), activation='relu', padding='same'))
#행렬 축소
model.add(MaxPooling2D(pool_size=(2,2)))
#모델 1차원 행렬로 변환
model.add(Flatten())
#128unit을 가지는 완전 연결 레이어
model.add(Dense(128, activation='relu'))
#이미지 역전파
model.add(Dense(num_classes, activation='softmax'))

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 32)        12832     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               200832    
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

In [25]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

In [28]:
callbacks = [EarlyStopping(monitor='val_accuracy', 
                           patience=2,                                                                       restore_best_weights=False),
                           ModelCheckpoint(filepath='best_model.h5',
                           monitor='val_accuracy',
                           save_best_only=True)]

In [31]:
model.fit(train_x, train_y,
          batch_size=500,
          epochs=10,
          verbose=1,
          validation_split=0.1,
          callbacks=callbacks)

Epoch 1/10
108/108 [==============================] - 22s 196ms/step - loss: 0.4595 - accuracy: 0.8723 - val_loss: 0.1215 - val_accuracy: 0.9672
Epoch 2/10
108/108 [==============================] - 21s 194ms/step - loss: 0.1078 - accuracy: 0.9677 - val_loss: 0.0739 - val_accuracy: 0.9785
Epoch 3/10
108/108 [==============================] - 21s 197ms/step - loss: 0.0700 - accuracy: 0.9790 - val_loss: 0.0529 - val_accuracy: 0.9853
Epoch 4/10
108/108 [==============================] - 21s 197ms/step - loss: 0.0537 - accuracy: 0.9837 - val_loss: 0.0479 - val_accuracy: 0.9877
Epoch 5/10
108/108 [==============================] - 22s 201ms/step - loss: 0.0454 - accuracy: 0.9862 - val_loss: 0.0439 - val_accuracy: 0.9863
Epoch 6/10
108/108 [==============================] - 22s 200ms/step - loss: 0.0392 - accuracy: 0.9880 - val_loss: 0.0372 - val_accuracy: 0.9888
Epoch 7/10
108/108 [==============================] - 22s 200ms/step - loss: 0.0335 - accuracy: 0.9900 - val_loss: 0.0377 - val_ac

In [32]:
score = model.evaluate(val_x, val_y, verbose=0)
print('Test loss:',score[0])
print('Test accuracy:',score[1])

Test loss: 0.028669198974967003
Test accuracy: 0.991100013256073
